In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install imbalanced-learn

In [ ]:
import scipy.io as sio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import scale,StandardScaler
from keras.layers import Dense, Input,Dropout
from tensorflow.keras.models import Sequential, model_from_json
from keras.models import Model
import os


path = "/content/drive/MyDrive/Watashara_Projects/8-Dengue/"
data_ = pd.read_csv(path + 'Feature_Selection/GLasso_feat.csv')
# csv_file = pd.read_csv(path+ 'Feature_Selection/GLasso_Train_feat.csv')
# deep_llm_feat_train = np.column_stack((port_feat_train, Train_BiGRU_1500))
data_np = np.array(data_)
data_scaled = scale(data_np[:, :])
print(np.shape(data_np))

label1=np.ones((554 ,1))#Value can be changed
label2=np.zeros((554 ,1))
label=np.append(label1,label2)
y = label

# y = labels


X_train, X_ind, y_train, y_ind = train_test_split(data_scaled, label, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_ind.shape)
print(y_train.shape)
print(y_ind.shape)

# X = np.array(X_train,dtype=float)
# y = y_train


# from imblearn.over_sampling import SMOTE, BorderlineSMOTE
# oversample = SMOTE()
# X_train, y_train = oversample.fit_resample(X_train, y_train)

# # print(X_train.shape)
# # print(X_ind.shape)
# # print(y_train.shape)
# # print(y_ind.shape)



# path = "/content/drive/MyDrive/Watashara_Projects/ACE/"
# train_data = pd.read_csv(path + 'selected_features_allfeat/XtrainData_wSMOTE.csv')


# X_train = train_data.iloc[:, :-1].values
# y_train = train_data.iloc[:, -1].values

# print(np.shape(X_train))

X = np.array(X_train,dtype=float)
[m1,n1] = np.shape(X)
# y = np.array(y_train)

# X =
y = y_train


fld_name = 'GLasso_DNN'

if not os.path.exists(path + (f'Results/{fld_name}/')):
    os.makedirs(path + f'Results/{fld_name}/')

[m,n]=np.shape(X)
sepscores = []
sepscores_ = []
ytest=np.ones((1,2))*0.5
yscore=np.ones((1,2))*0.5
def get_con_model():
    input_1 = Input(shape=(n,), name='Protein')
    protein_input1 = Dense(int(n*2), activation='relu', name='High_dim_feature_1')(input_1)
    protein_input1=Dropout(0.5)(protein_input1)
    protein_input1 = Dense(int(n), activation='relu', name='High_dim_feature_2')(protein_input1)
    protein_input1=Dropout(0.5)(protein_input1)
    # Apply Self-Attention Layer
    attention_output = SelfAttention(attention_width=50)(protein_input1)
    # protein_input1 = Dense(int(n/2), activation='relu',  name='High_dim_feature_3')(protein_input1)
    # protein_input1=Dropout(0.5)(protein_input1)
    output = Dense(int(n/4), activation='relu',  name='High_dim_feature')(protein_input1)
    outputs = Dense(2, activation='softmax', name='output')(output)
    model = Model(inputs= input_1, outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model

model=get_con_model()

skf= StratifiedKFold(n_splits=5)
for i, (train, test) in enumerate (skf.split(X,y)):
    y_train=to_categorical(y[train])
    cv_clf =get_con_model()
    hist=cv_clf.fit(X[train],
                    y_train,
                    epochs=15)
    y_test=to_categorical(y[test])
    ytest=np.vstack((ytest,y_test))
    y_test_tmp=y[test]
    y_score=cv_clf.predict(X[test])
    yscore=np.vstack((yscore,y_score))
    fpr, tpr, _ = roc_curve(y_test[:,0], y_score[:,0])
    roc_auc = auc(fpr, tpr)
    y_class= categorical_probas_to_classes(y_score)
    acc, precision,npv, sensitivity, specificity, mcc,f1 = calculate_performace(len(y_class), y_class, y_test_tmp)
    sepscores.append([acc, precision,npv, sensitivity, specificity, mcc,f1,roc_auc])
    print('DNN:acc=%f,precision=%f,npv=%f,sensitivity=%f,specificity=%f,mcc=%f,f1=%f,roc_auc=%f'
          % (acc, precision,npv, sensitivity, specificity, mcc,f1, roc_auc))
    cv_clf.save(path + f'Results/{fld_name}/' + str(i) + 'DNN_model.keras')


scores=np.array(sepscores)
print("acc=%.2f%% (+/- %.2f%%)" % (np.mean(scores, axis=0)[0]*100,np.std(scores, axis=0)[0]*100))
print("precision=%.2f%% (+/- %.2f%%)" % (np.mean(scores, axis=0)[1]*100,np.std(scores, axis=0)[1]*100))
print("npv=%.2f%% (+/- %.2f%%)" % (np.mean(scores, axis=0)[2]*100,np.std(scores, axis=0)[2]*100))
print("sensitivity=%.2f%% (+/- %.2f%%)" % (np.mean(scores, axis=0)[3]*100,np.std(scores, axis=0)[3]*100))
print("specificity=%.2f%% (+/- %.2f%%)" % (np.mean(scores, axis=0)[4]*100,np.std(scores, axis=0)[4]*100))
print("mcc=%.2f%% (+/- %.2f%%)" % (np.mean(scores, axis=0)[5]*100,np.std(scores, axis=0)[5]*100))
print("f1=%.2f%% (+/- %.2f%%)" % (np.mean(scores, axis=0)[6]*100,np.std(scores, axis=0)[6]*100))
print("roc_auc=%.2f%% (+/- %.2f%%)" % (np.mean(scores, axis=0)[7]*100,np.std(scores, axis=0)[7]*100))
result1=np.mean(scores,axis=0)
H1=result1.tolist()
sepscores.append(H1)
result=sepscores
row=yscore.shape[0]
yscore=yscore[np.array(range(1,row)),:]
yscore_sum = pd.DataFrame(data=yscore)
yscore_sum.to_csv(path + f'Results/{fld_name}/DNN_yscore.csv')
ytest=ytest[np.array(range(1,row)),:]
ytest_sum = pd.DataFrame(data=ytest)
ytest_sum.to_csv(path + f'Results/{fld_name}/DNN_ytest.csv')

data_csv = pd.DataFrame(data=result)
data_csv.to_csv(path +f'Results/{fld_name}/DNN_CV_results.csv')


(1108, 279)
(886, 279)
(222, 279)
(886,)
(222,)
Epoch 1/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5955 - loss: 0.7375
Epoch 2/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7838 - loss: 0.4530
Epoch 3/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8409 - loss: 0.3383
Epoch 4/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9006 - loss: 0.2450
Epoch 5/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9092 - loss: 0.2330
Epoch 6/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9181 - loss: 0.1697
Epoch 7/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9558 - loss: 0.1218
Epoch 8/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9600 - loss: 0.1311 
Epoch 9/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9471 - loss: 0.1291
Epoch 10/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9589 - loss: 0.1241
Epoch 11/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9596 - loss: 0.1154
Epoch 12/15
23/23 ━━━━

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
DNN:acc=0.802260,precision=0.833333,npv=0.777778,sensitivity=0.747126,specificity=0.855556,mcc=0.606882,f1=0.787879,roc_auc=0.902618
Epoch 1/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5837 - loss: 0.7534
Epoch 2/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7733 - loss: 0.5159
Epoch 3/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8965 - loss: 0.3000
Epoch 4/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8789 - loss: 0.2770
Epoch 5/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9154 - loss: 0.2013
Epoch 6/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9250 - loss: 0.1798
Epoch 7/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9241 - loss: 0.1834
Epoch 8/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9544 - loss: 0.1333
Epoch 9/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9400 - loss: 0.1957
Epoch 10/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9576 - 

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
DNN:acc=0.830508,precision=0.778846,npv=0.904110,sensitivity=0.920455,specificity=0.741573,mcc=0.672410,f1=0.843750,roc_auc=0.918539
Epoch 1/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5609 - loss: 0.7723
Epoch 2/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7539 - loss: 0.5227
Epoch 3/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8417 - loss: 0.3379
Epoch 4/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8494 - loss: 0.3416
Epoch 5/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9031 - loss: 0.2539
Epoch 6/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9498 - loss: 0.1549
Epoch 7/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9278 - loss: 0.1757
Epoch 8/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9322 - loss: 0.1482
Epoch 9/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9605 - loss: 0.0918
Epoch 10/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9324 - l

**Indpendent test**